In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors
import numpy as np
from tqdm import tqdm

In [ ]:
data_prefix = 'data_cleaned/'
df = pd.read_csv(data_prefix + 'original_data.csv')
df

,MOL_1,MOL_2,logV,MolFrac_1,T,benchmark,type,combos
0,B(Br)(Br)Br,CCO,-0.140112,1.0,298.00,train,original,B(Br)(Br)Br-CCO
1,BrBr,CCO,-0.011187,1.0,298.00,train,original,BrBr-CCO
2,BrC(Br)(Br)Br,c1ccccc1,-0.219755,0.0,298.15,train,original,BrC(Br)(Br)Br-c1ccccc1
3,BrC(Br)(Br)Br,c1ccccc1,-0.080765,0.1,298.15,train,original,BrC(Br)(Br)Br-c1ccccc1
4,BrC(Br)(Br)Br,c1ccccc1,0.050070,0.2,298.15,train,original,BrC(Br)(Br)Br-c1ccccc1
...,...,...,...,...,...,...,...,...
81805,CCCO,c1ccncc1,-0.055304,0.4,308.15,test,expanded,CCCO-c1ccncc1
81806,CCCO,c1ccncc1,-0.073118,0.3,308.15,test,expanded,CCCO-c1ccncc1
81807,CCCO,c1ccncc1,-0.082963,0.2,308.15,test,expanded,CCCO-c1ccncc1
81808,CCCO,c1ccncc1,-0.091413,0.1,308.15,test,expanded,CCCO-c1ccncc1


In [10]:
unique_smiles_1 = df['MOL_1'].unique()
unique_smiles_2 = df['MOL_2'].unique()
all_unique_smiles = pd.Series(list(set(unique_smiles_1) | set(unique_smiles_2)))

print(f"MOL_1 unique SMILES: {len(unique_smiles_1)}")
print(f"MOL_2 unique SMILES: {len(unique_smiles_2)}")
print(f"Total unique SMILES: {len(all_unique_smiles)}")

df_feats = pd.DataFrame({'SMILES': all_unique_smiles})
df_feats

MOL_1 unique SMILES: 1753
MOL_2 unique SMILES: 1753
Total unique SMILES: 1753


,SMILES
0,C/C=C\CCC
1,OCCC(C)C
2,CC(C)CC(=O)CC(C)C
3,CC(C)OCC(C)C
4,CC(C)CCCCC(C)C
...,...
1748,CC(C)(C)SCC
1749,OCCO
1750,OOC(=O)C
1751,CCCC(OCCCC)=O


In [13]:
# 计算所有RDKit特征的函数
def calculate_rdkit_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    
    features = {}
    
    for desc_name, desc_func in Descriptors.descList:
        try:
            features[desc_name] = desc_func(mol)
        except:
            features[desc_name] = np.nan
    return features

In [14]:
print(f"Calculating rdkit features for {len(df_feats)} SMILES...")

all_feature_names = [desc_name for desc_name, _ in Descriptors.descList]
print(f"In total {len(all_feature_names)} Rdkit descriptors")

features_list = []
failed_smiles = []

for idx, smiles in tqdm(enumerate(df_feats['SMILES']), total=len(df_feats)):
    features = calculate_rdkit_features(smiles)
    if features is not None:
        features_list.append(features)
    else:
        failed_smiles.append(smiles)
        features_list.append({key: np.nan for key in all_feature_names})


features_df = pd.DataFrame(features_list)
df_feats = pd.concat([df_feats, features_df], axis=1)

print(f"\nCompleted")
print(f"Success: {len(df_feats) - len(failed_smiles)}")
print(f"Failure: {len(failed_smiles)}")
if failed_smiles:
    print(f"Failed SMILES: {failed_smiles[:5]}...")  

df_feats

Calculating rdkit features for 1753 SMILES...
In total 208 Rdkit descriptors


100%|██████████| 1753/1753 [00:11<00:00, 147.97it/s]



Completed
Success: 1753
Failure: 0


,SMILES,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,C/C=C\CCC,2.180556,1.232639,2.180556,1.232639,0.450423,84.162,72.066,84.093900,36,...,0,0,0,0,0,0,0,0,0,0
1,OCCC(C)C,8.236667,0.331019,8.236667,0.331019,0.534089,88.150,76.054,88.088815,38,...,0,0,0,0,0,0,0,0,0,0
2,CC(C)CC(=O)CC(C)C,11.063889,0.407407,11.063889,0.407407,0.589429,142.242,124.098,142.135765,60,...,0,0,0,0,0,0,0,0,0,0
3,CC(C)OCC(C)C,5.303241,0.386481,5.303241,0.386481,0.548153,116.204,100.076,116.120115,50,...,0,0,0,0,0,0,0,0,0,0
4,CC(C)CCCCC(C)C,2.300967,0.895521,2.300967,0.895521,0.511263,142.286,120.110,142.172151,62,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1748,CC(C)(C)SCC,2.237269,0.467014,2.237269,0.467014,0.509606,118.245,104.133,118.081621,44,...,1,0,0,0,0,0,0,0,0,0
1749,OCCO,7.625000,-0.125000,7.625000,0.125000,0.407528,62.068,56.020,62.036779,26,...,0,0,0,0,0,0,0,0,0,0
1750,OOC(=O)C,9.340278,-0.689815,9.340278,0.689815,0.326608,76.051,72.019,76.016044,30,...,0,0,0,0,0,0,0,0,0,0
1751,CCCC(OCCCC)=O,10.695652,-0.059306,10.695652,0.059306,0.436174,144.214,128.086,144.115030,60,...,0,0,0,0,0,0,0,0,1,0


In [17]:
df_merged = df.merge(
    df_feats.add_prefix('MOL1_'), 
    left_on='MOL_1', 
    right_on='MOL1_SMILES',
    how='left'
)

df_merged = df_merged.merge(
    df_feats.add_prefix('MOL2_'),
    left_on='MOL_2',
    right_on='MOL2_SMILES',
    how='left'
)

df_merged = df_merged.drop(columns=['MOL1_SMILES', 'MOL2_SMILES'])

print(f"original data shape: {df.shape}")
print(f"data_feature shape: {df_merged.shape}")
print(f"number of features added: {df_merged.shape[1] - df.shape[1]}")

original data shape: (81810, 8)
data_feature shape: (81810, 424)
number of features added: 416


In [18]:
df_merged

,MOL_1,MOL_2,logV,MolFrac_1,T,benchmark,type,combos,MOL1_MaxEStateIndex,MOL1_MinEStateIndex,...,MOL2_fr_sulfide,MOL2_fr_sulfonamd,MOL2_fr_sulfone,MOL2_fr_term_acetylene,MOL2_fr_tetrazole,MOL2_fr_thiazole,MOL2_fr_thiocyan,MOL2_fr_thiophene,MOL2_fr_unbrch_alkane,MOL2_fr_urea
0,B(Br)(Br)Br,CCO,-0.140112,1.0,298.00,train,original,B(Br)(Br)Br-CCO,3.104167,0.270833,...,0,0,0,0,0,0,0,0,0,0
1,BrBr,CCO,-0.011187,1.0,298.00,train,original,BrBr-CCO,2.750000,2.750000,...,0,0,0,0,0,0,0,0,0,0
2,BrC(Br)(Br)Br,c1ccccc1,-0.219755,0.0,298.15,train,original,BrC(Br)(Br)Br-c1ccccc1,3.125000,-0.250000,...,0,0,0,0,0,0,0,0,0,0
3,BrC(Br)(Br)Br,c1ccccc1,-0.080765,0.1,298.15,train,original,BrC(Br)(Br)Br-c1ccccc1,3.125000,-0.250000,...,0,0,0,0,0,0,0,0,0,0
4,BrC(Br)(Br)Br,c1ccccc1,0.050070,0.2,298.15,train,original,BrC(Br)(Br)Br-c1ccccc1,3.125000,-0.250000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81805,CCCO,c1ccncc1,-0.055304,0.4,308.15,test,expanded,CCCO-c1ccncc1,7.875000,0.319444,...,0,0,0,0,0,0,0,0,0,0
81806,CCCO,c1ccncc1,-0.073118,0.3,308.15,test,expanded,CCCO-c1ccncc1,7.875000,0.319444,...,0,0,0,0,0,0,0,0,0,0
81807,CCCO,c1ccncc1,-0.082963,0.2,308.15,test,expanded,CCCO-c1ccncc1,7.875000,0.319444,...,0,0,0,0,0,0,0,0,0,0
81808,CCCO,c1ccncc1,-0.091413,0.1,308.15,test,expanded,CCCO-c1ccncc1,7.875000,0.319444,...,0,0,0,0,0,0,0,0,0,0


In [16]:
df_merged.to_csv(data_prefix + 'all_data_with_features.csv', index=False)